In [ ]:
# binarização 
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Exemplo de transformer customizado para imputação segmentada
class GroupMedianImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_col, target_cols):
        self.group_col = group_col
        self.target_cols = target_cols
        self.group_medians = None

    def fit(self, X, y=None):
        # Calcula a mediana por grupo
        self.group_medians = (
            X.groupby(self.group_col)[self.target_cols]
            .median()
            .to_dict()
        )
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.target_cols:
            for group, median in self.group_medians[col].items():
                mask = (X[self.group_col] == group) & (X[col].isna())
                X.loc[mask, col] = median
        return X

# Colunas
num_cols = ['renda_mensal', 'comprometimento_renda', 'utilizacao_credito', 'divida_ratio']
cat_cols = ['faixa_etaria']

# Pipeline para imputação segmentada em renda e comprometimento
segment_imputer = GroupMedianImputer(group_col='faixa_etaria',
                                     target_cols=['renda_mensal', 'comprometimento_renda'])

# Imputação simples para outras numéricas
num_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Imputação categórica
cat_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='não informado')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, ['utilizacao_credito', 'divida_ratio']),
        ('cat', cat_imputer, cat_cols)
    ],
    remainder='passthrough'
)

# Pipeline completo
pipeline = Pipeline(steps=[
    ('segment_imputer', segment_imputer),   # imputação segmentada
    ('preprocessor', preprocessor)          # imputação simples + encoding
])

# Exemplo de uso
df_transformed = pipeline.fit_transform(df_raw)

print("Shape original:", df_raw.shape)
print("Shape transformado:", df_transformed.shape)


In [ ]:
df_transformed

In [ ]:

# 1. Executa o pipeline
df_array = pipeline.fit_transform(df_raw)

# 2. Recupera os nomes das colunas geradas pelo preprocessor
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# 3. Cria DataFrame com essas colunas
df_transformed = pd.DataFrame(df_array, columns=feature_names)

# 4. Cria manualmente as flags de missing a partir do df_raw
flags = pd.DataFrame({
    'renda_mensal_missing': df_raw['renda_mensal'].isna().astype(int),
    'comprometimento_renda_missing': df_raw['comprometimento_renda'].isna().astype(int)
}, index=df_raw.index)
df_final = pd.concat([df_transformed, flags], axis=1)
# descartar as colunas originais com valores nulos.
df_final = df_final.drop(columns=['remainder__renda_mensal', 'remainder__comprometimento_renda'])


df_final.isnull().sum()



In [ ]:
#from scipy.stats.mstats import winsorize

# Copiar a coluna para não alterar o original
#renda = df['renda_mensal'].copy()

# Winsorization: corta 1% dos valores mais baixos e 1% dos mais altos
# Ajuste os limites conforme necessidade (ex: 0.01 = 1%)
#renda_wins = winsorize(renda, limits=[0.05, 0.0]) 

# Substituir no dataframe
#df['renda_mensal_wins'] = renda_wins

# Comparar estatísticas antes e depois
#print("Antes:\n", df['renda_mensal'].describe())
#print("\nDepois:\n", df['renda_mensal_wins'].describe())
df_transformed.sample(20)

In [ ]:
# comparando dados original, sem outliers(abaixo de 50.000),e escala logaritmica
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Original
sns.histplot(df['comprometimento_renda'], bins=30, kde=True, ax=axes[0])
axes[0].set_title("Original")

# 2. Sem outliers (exemplo: abaixo de 50 mil)
sns.histplot(df[df['comprometimento_renda'] < 50000]['comprometimento_renda'], bins=30, kde=True, ax=axes[1])
axes[1].set_title("Sem Outliers (<50k)")

# 3. Escala logarítmica
sns.histplot(df['comprometimento_renda'], bins=30, kde=True, log_scale=True, ax=axes[2])
axes[2].set_title("Escala Logarítmica")

plt.tight_layout()
plt.show()